author: Samuel Begg

date: 06/04/2024

Compute Fisher information for a periodically driven XY model, as discussed in Mishra, Bayat, Phys. Rev. Lett. 127, 080504 (2021)

1. Define simulation parameters

In [13]:
import methods
import numpy as np
import scipy.linalg as lin
from datetime import datetime
import copy
from joblib import Parallel, delayed, load, dump
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

stem0 = '/Users/samuelbegg/'
stem = stem0  + 'Documents/Sensing/'

# System parameters for periodic boundary condition spin chain
J = 1.0  # +ve is Ferromagnet [Hamiltonian as defined Mishra, Bayat, PRL 127, 080504 (2021)]
gamma = 1.0
sites = 100
h1_amp = 1.5
h1_period = 1.0 #period of the drive

# Initial state
initial_state = 'ground_state' 
# 'ground_state': starts in the system ground-state. 
# 'flips': manually flips specific spins (with the rest down), see below. Default: single flip. Odd number of excitations (spin) = even parity (fermions) = PBC. 
# 'momentum': a k = 0 momentum state. Even parity = PBC.
# 'spin_up': all spins up, odd number excitations (spin) = even parity = PBC, even number excitations (spin) = odd parity (fermions) = ABC

# Boundary conditions
boundary_conditions = 'PBC' 
# 'PBC' = periodic boundary conditions or 'ABC' = anti-periodic boundary conditions
# The boundary conditions must reflect the fermion parity in the initial conditions (see above).
# The Fourier transform definition is automatically adjusted based on number sites and boundary conditions, as in arXiv:1707.02400.

# Sensing parameters
phasepoints = 1 # number of points for calculating h0
h0mat = np.linspace(0.9, 0.9, phasepoints) #+ 0.001 * np.ones(phasepoints) # range of h0 points to sample
sub_system_range = np.arange(20,20+1) # range of subsystem sizes for evaluating Fisher information

# Time and integration parameters
dt = 0.01 # time step data is saved at
times = 100 # number of steps 
tvec = dt * np.arange(1,times+1) # time vector for output
method = 'RK45' # integration method #'RK45' (4th order), 'DOP853' (8th order), 'BDF'
rtol = 10**(-12) # relative tolerance of integration scheme
atol = 10**(-14) # absolute tolerance of integration scheme

# Fisher Information Computation
tol = 10**(-8) # when evaluating the Fisher information we don't consider singular terms for which |w[rr] + w[ss]| < tol 
shift = 10**(-7) # finite division size for calculating derivative of reduced density matrix, i.e. drho/dh0 ~ (rho(h0) - rho(h0 + shift))/shift for the order2 version
derivative_estimator = 'order4' # 2nd order = 'order2' or 4th order 'order4' approximation for the derivative of reduced density matrix
num_cores = -1  # number of cores to parallelize with
save_results = 'save' # 'save' or 'bin' 

# Initialize periodic field
h1 = h1_amp*np.sin(2*np.pi/(h1_period)*dt*np.arange(0,times+1)) # initialize periodic field, note + 1 time added here for RK4 and heun integration step endpoints


2. Initialize correlation matrices <c^{dag}_i c_j> and <c^{dag}_i c^{dag}_j>


In [14]:
obs, Dag_obs = methods.initialize_state(sites, gamma, J, h0mat[0], h1[0], boundary_conditions, initial_state)

3. Perform time-evolution for a range of h_0

In [15]:
Fisher_mat = []
particle_numberL = []
particle_numberR = []
pair_creationL = []
corrmatL = []

#loop over h0
output = Parallel(n_jobs=num_cores)(delayed(methods.Fisher_Calc_eff)(phase, obs, Dag_obs, J, gamma, h0mat[phase], shift, h1, times, dt, sites, sub_system_range, tol, derivative_estimator, method, initial_state, boundary_conditions, atol, rtol) for phase in range(0,phasepoints))

#collect output of parallel computation
for kk in range(0,phasepoints):
    
    data = output[kk]

    Fisher_mat = Fisher_mat + [data[0]]

    particle_numberL = particle_numberL + [data[1]]

    corrmatL = corrmatL + [data[2]]

    pair_creationL = pair_creationL + [data[3]]

    
#Calculate time average over 2nd half of time evolution to get steady state Fisher information

#initialize variables
time_tot = np.size(Fisher_mat[0][0,:])

Fisher_av = np.zeros([phasepoints,np.size(sub_system_range)])

#loop over h0 and subsystem size
for kk in range(0, phasepoints):

    for ss in range(0, np.size(sub_system_range)):
        #Extract steady state (or late time) Fisher information by averaging over final 10% of times
        Fisher_av[kk,ss] = Fisher_av[kk,ss] + np.mean(Fisher_mat[kk][ss,int(9*time_tot/10):int(time_tot)])
        

0 h0 value =  0.9
0:00:15.022359 End Simulation
0:00:15.023499 End Simulation
0:00:15.394973 End Simulation
0:00:15.342646 End Simulation
0:00:14.867004 End Simulation
Simulations finished, reduced density matrices extracted. Now calculate Fisher information.
Analyse sub_system L = 20


/Users/samuelbegg/Documents/Sensing/methods.py:1208: ComplexWarning: Casting complex values to real discards the imaginary part
  Fisher[zzz,ttt] = Fisher_time


Subsystem analaysis complete. Number of avoided divergences (average per time) = 298
0:00:00.735370 End Fisher Calculation


4. Save Results

In [16]:
measure_tvec = dt * np.arange(0,len(Fisher_mat[0][0][:]))

rtol = 10
if save_results == 'save':
    
    np.save('Results/h0mat.npy',h0mat)
    np.save('Results/measure_tvec.npy',measure_tvec)
    np.save('Results/Fisher_mat.npy',Fisher_mat)
    np.save('Results/density.npy',particle_numberL)
    np.save('Results/pair_creation.npy',pair_creationL)
    np.save('Results/corrmat.npy',corrmatL)
    np.save('Results/subsystem_range.npy',sub_system_range)

    np.save('Results/h0mat_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',h0mat)
    np.save('Results/measure_tvec_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',measure_tvec)
    np.save('Results/Fisher_mat_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',Fisher_mat)
    np.save('Results/density_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',particle_numberL)
    np.save('Results/pair_creation_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',pair_creationL)
    np.save('Results/corrmat_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',corrmatL)
    np.save('Results/subsystem_range_100_test_h_0.9_subsystem_'+str(sub_system_range[0])+'.npy',sub_system_range)